# LLM Iteration 2

**Aim**

Add complexity to the previous model by having the patients stay a set length of time on the ward, which will be constant and the same for all patient types. The patient will hold the bed during this time so no more patients can be admitted to that bed until they leave, creating the possibility of a queue. 

**Prompt**

Code a discrete-event simulation model of a stroke ward in a hospital in python 3.10 and simpy 4.
Please code the full model specified. Do not return a simplified version. Show all code. 

All time units in the model are in days.
Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.

Patients to arrive in a stroke ward that admits four different patient types: Stroke, TIA, Complex Neurological or Other. Each patient type had a different interarrival time distribution. There are 10 beds on the stroke ward. The patient should only be admitted if there is an available bed. The patient holds the bed until their length of stay has elapsed. The length of stay for all patients is 7 days. After the length of stay has elapsed, the patient leaves the model. 

All patient types must have their own generator function. 

The model should print out useful information after each event. 
The model should include a user settable run length. This defaults to 2 months. 

The interarrival time distribution and parameters of patients are dependent on the patient type. This describes the average time between required admissions. For each distribution time is measured in days. 
Stroke = Exponential: 1.2
TIA = Exponential: 9.3
Complex Neurological = Exponential: 3.6
Other = Exponential: 3.2

Numpy should be used for sampling. Each interarrival distribution should have its own numpy.random.Generator object.

Provide sources.


**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

Sources:
1. SimPy Documentation: https://simpy.readthedocs.io/en/latest/
2. NumPy Documentation: https://numpy.org/doc/stable/

In [2]:
import simpy
import numpy as np

In [3]:
# Define the parameters
NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 60  # Default run length in days (2 months)
INTERARRIVAL_TIMES = {
    'Stroke': 1.2,
    'TIA': 9.3,
    'Complex Neurological': 3.6,
    'Other': 3.2
}

In [4]:
class StrokeWard:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=NUM_BEDS)
        self.patient_count = 0
        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        ##################################

    def admit_patient(self, patient):
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            ##################################
            
            print(f'Patient {patient.id} of type {patient.type} admitted at '
                  f'day {round(self.env.now, 2)}')
            yield self.env.timeout(LENGTH_OF_STAY)
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(bed_leave_time - bed_arrival_time)
            ##################################
            print(f'Patient {patient.id} of type {patient.type} discharged at '
                  f'day {round(self.env.now, 2)}')

class Patient:
    def __init__(self, id, type):
        self.id = id
        self.type = type

def patient_generator(env, ward):
    while True:
        interarrival_time = np.random.default_rng().\
                            exponential(INTERARRIVAL_TIMES['Stroke'])
        yield env.timeout(interarrival_time)
        patient_type = np.random.choice(list(INTERARRIVAL_TIMES.keys()))
        patient = Patient(ward.patient_count, patient_type)
        ward.patient_count += 1
        env.process(ward.admit_patient(patient))

In [5]:
# Setup and start the simulation
env = simpy.Environment()
ward = StrokeWard(env)
env.process(patient_generator(env, ward))
env.run(until=RUN_LENGTH)

Patient 0 of type Complex Neurological admitted at day 0.45
Patient 1 of type Other admitted at day 2.92
Patient 2 of type Other admitted at day 4.58
Patient 3 of type Stroke admitted at day 4.79
Patient 4 of type TIA admitted at day 4.96
Patient 5 of type Stroke admitted at day 5.79
Patient 6 of type TIA admitted at day 6.64
Patient 0 of type Complex Neurological discharged at day 7.45
Patient 7 of type Complex Neurological admitted at day 8.78
Patient 1 of type Other discharged at day 9.92
Patient 8 of type Other admitted at day 10.34
Patient 9 of type TIA admitted at day 11.04
Patient 10 of type Stroke admitted at day 11.08
Patient 11 of type Complex Neurological admitted at day 11.19
Patient 2 of type Other discharged at day 11.58
Patient 3 of type Stroke discharged at day 11.79
Patient 12 of type Complex Neurological admitted at day 11.89
Patient 4 of type TIA discharged at day 11.96
Patient 5 of type Stroke discharged at day 12.79
Patient 13 of type Stroke admitted at day 13.33
P

**Checks**

In [11]:
class TestCase:
    def __init__(self, env, ward):
        self.env = env
        self.ward = ward
        self.env.process(patient_generator(self.env, self.ward))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        self.env.run(until=RUN_LENGTH)

        for los in ward.los_store:
            if los > LENGTH_OF_STAY:
                print('Patient length of stay exceeded limit.')
            elif los < LENGTH_OF_STAY:
                print('Patient did not stay for the full duration.')

In [7]:
# Checks run length
NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 10

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_run_length()

Patient 0 of type TIA admitted at day 1.36
Patient 1 of type Other admitted at day 2.23
Patient 2 of type Other admitted at day 4.81
Patient 3 of type TIA admitted at day 5.4
Patient 4 of type Other admitted at day 5.77
Patient 0 of type TIA discharged at day 8.36
Patient 1 of type Other discharged at day 9.23


In [8]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
LENGTH_OF_STAY = 7
RUN_LENGTH = 60

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_bed_hold()

Patient 0 of type Stroke admitted at day 4.73
Patient 0 of type Stroke discharged at day 11.73
Patient 1 of type Complex Neurological admitted at day 11.73
Patient 1 of type Complex Neurological discharged at day 18.73
Patient 2 of type Complex Neurological admitted at day 18.73
Patient 2 of type Complex Neurological discharged at day 25.73
Patient 3 of type Stroke admitted at day 25.73
Patient 3 of type Stroke discharged at day 32.73
Patient 4 of type TIA admitted at day 32.73
Patient 4 of type TIA discharged at day 39.73
Patient 5 of type Stroke admitted at day 39.73
Patient 5 of type Stroke discharged at day 46.73
Patient 6 of type TIA admitted at day 46.73
Patient 6 of type TIA discharged at day 53.73
Patient 7 of type Complex Neurological admitted at day 53.73


In [9]:
# Checks if stay length correctly implemented
NUM_BEDS = 10
LENGTH_OF_STAY = 50
RUN_LENGTH = 60

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_length_of_stay()

Patient 0 of type Complex Neurological admitted at day 0.59
Patient 1 of type Complex Neurological admitted at day 0.83
Patient 2 of type Other admitted at day 1.19
Patient 3 of type Stroke admitted at day 1.79
Patient 4 of type Other admitted at day 3.88
Patient 5 of type Stroke admitted at day 4.68
Patient 6 of type TIA admitted at day 4.95
Patient 7 of type TIA admitted at day 5.12
Patient 8 of type Other admitted at day 5.22
Patient 9 of type Other admitted at day 8.45
Patient 0 of type Complex Neurological discharged at day 50.59
Patient 10 of type Complex Neurological admitted at day 50.59
Patient 1 of type Complex Neurological discharged at day 50.83
Patient 11 of type Stroke admitted at day 50.83
Patient 2 of type Other discharged at day 51.19
Patient 12 of type Stroke admitted at day 51.19
Patient 3 of type Stroke discharged at day 51.79
Patient 13 of type Complex Neurological admitted at day 51.79
Patient 4 of type Other discharged at day 53.88
Patient 14 of type TIA admitted

**Reflection**

The model produced was able to admit patients and assign them a bed. The bed is held until the patient leaves, which was checked by providing only one bed and therefore only one patient could attend at that time. The length of stay could be changed and this impacted on how many patients were admitted as other patients were holding a bed for a longer period of time. 

It only used the mean interarrival time for Stroke patients (```np.random.default_rng().exponential(INTERARRIVAL_TIMES['Stroke'])```) despite specifying that each patient type has its own arrival time mean. The model does not have the option for reproducibility yet and outputs different results each time the code is run.